1. Bajar el CSV de Oferta Gastronómica desde Buenos Aires Data (https://data.buenosaires.gob.ar/dataset/). Idealmente hacer esto con Python.

In [1]:
import pandas as pd
import pymysql


Creamos un DataFrame de pandas con el csv.

In [22]:
df = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')

In [4]:
df.head(3)

,long,lat,id,nombre,categoria,cocina,ambientacion,telefono,mail,horario,calle_nombre,calle_altura,calle_cruce,direccion_completa,barrio,comuna,codigo_postal,codigo_postal_argentino
0,-58.363307,-34.622963,1602,GUANTANAMERA,RESTAURANTE,CUBANA,MODERNO,4314 0770,NaN,NaN,MOREAU DE JUSTO ALICIA,2092,NaN,"MOREAU DE JUSTO, ALICIA AV. 2092",Puerto Madero,Comuna 1,1107.0,C1107AFP
1,-58.392330,-34.621554,676,1843 VINACOTECA,VINERIA,NaN,NaN,4779 0806,NaN,lunes a domingo de 8 a 00,HUMBERTO 1º,1843,NaN,HUMBERTO 1º 1843,San Cristobal,Comuna 3,NaN,NaN
2,-58.453542,-34.560311,487,2080 EXPRESS,RESTAURANTE,TRADICIONAL,CLASICO PORTEÑO,4788 4500,NaN,lunes a domingo de 8 a 00,JURAMENTO,2080,NaN,JURAMENTO AV. 2080,Belgrano,Comuna 13,1428.0,C1428DNH


2. Crear una tabla con los siguientes campos: id_local, nombre, categoria, direccion, barrio, comuna, para posteriormente poblarla con los datos bajados, utilzando el conector desde el script de Python.

Voy a crear una conexión con MySQL y en database no voy a poner nada, para después crear una nueva base de datos.

In [5]:
conexion = pymysql.connect(host='localhost', user='#####', password='#####', database='')

Se conectó bien. Ahora voy a crear una base de datos nueva.

In [8]:
query1 = 'CREATE DATABASE Nueva'  # Instrucción para crear la nueva base de datos (igual que en SQL).
c = conexion.cursor()  # El método cursor() de la instancia conexion sirve para ejecutar las consultas.
c.execute(query1)



1

Indica que se ejecutó bien. Verifico en MySQL y está la base de datos nueva.

Para evitar errores, actualizo conexion y conexion.cursor() con la nueva db.

In [9]:
conexion = pymysql.connect(host='localhost', user='#####', password='#####', database='Nueva')

In [10]:
c = conexion.cursor()

Ahora creo la nueva tabla (igual que en SQL).

In [20]:
query2 = """CREATE TABLE Locales(idLocal INT NOT NULL PRIMARY KEY,  # Si no ordenamos el DataFrame y lo queremos pasar 
                                                                    # tal como está, no ponemos AUTO_INCREMENT
								nombre VARCHAR(50),
                                categoria VARCHAR(20),
                                direccion VARCHAR(50),
                                barrio VARCHAR(30),
                                comuna VARCHAR(15))"""  # Con comillas triples, porque está en varias líneas.


c.execute(query2)


0

Indica que se ejecutó bien. Verifico en MySQL y se creó.

Si revisamos bien todo el df, hay valores que son NaN. Para evitar errores, los reemplazo.

In [49]:
df = df.fillna('N/A')

Ahora hay que insertar los valores en la tabla:

In [26]:
for i in range(len(df)):
    valores = (df.loc[i, 'id'], df.loc[i, 'nombre'], df.loc[i, 'categoria'], df.loc[i, 'direccion_completa'], 
               df.loc[i, 'barrio'], df.loc[i, 'comuna'])
    query3 = 'INSERT INTO Locales(idLocal, nombre, categoria, direccion, barrio, comuna) VALUES(%s, %s, %s, %s, %s, %s)'
    c.execute(query3, valores)

conexion.commit()

Se insertaron correctamente.

Cerramos la conexión.

In [28]:
conexion.close()

3. A partir de tener los datos disponibles, responder a las siguientes preguntas:

   a) ¿Cuál es el barrio con mayor cantidad de Pubs?

 

Volvemos a conectarnos.

In [29]:
conexion = pymysql.connect(host='localhost', user='#####', password='#####', database='Nueva')
c = conexion.cursor()

In [41]:
query4 = 'SELECT barrio, COUNT(*) FROM Locales WHERE categoria = "pub" GROUP BY barrio ORDER BY 2 DESC'
c.execute(query4)
for fila in c:
    print(fila)

('Recoleta', 8)
('Caballito', 6)
('Villa Devoto', 5)
('Flores', 4)
('Mataderos', 4)
('Retiro', 4)
('Palermo', 3)
('San Nicolas', 3)
('Balvanera', 2)
('Liniers', 2)
('NuÃƒÂ±ez', 2)
('Villa Del Parque', 2)
('Agronomia', 2)
('Villa Ortuzar', 2)
('Monserrat', 1)
('Villa Santa Rita', 1)
('Villa Lugano', 1)
('Villa Pueyrredon', 1)


Recoleta.

  b) Obtener la cantidad de locales por categoría
   

In [43]:
query5 = 'SELECT categoria, COUNT(*) FROM Locales GROUP BY categoria ORDER BY 2 DESC'
c.execute(query5)
for fila in c:
    print(fila)

('RESTAURANTE', 1990)
('CAFE', 334)
('BAR', 247)
('N/A', 116)
('CONFITERIA', 55)
('PUB', 53)
('VINERIA', 22)
('SANDWICHERIA', 4)
('DELIVERY & TAKE AWAY', 2)


c) Obtener la cantidad de restaurantes por comuna

In [47]:
query6 = 'SELECT  comuna, COUNT(*) FROM Locales WHERE categoria = "RESTAURANTE" GROUP BY comuna ORDER BY 2 DESC'
c.execute(query6)
for fila in c:
    print(fila)

('Comuna 1', 609)
('Comuna 14', 366)
('Comuna 2', 227)
('Comuna 3', 145)
('Comuna 13', 108)
('Comuna 4', 95)
('Comuna 6', 87)
('Comuna 7', 64)
('Comuna 15', 57)
('Comuna 5', 56)
('Comuna 9', 52)
('Comuna 10', 43)
('Comuna 12', 42)
('Comuna 11', 33)
('Comuna 8', 6)


Importante: siempre cerrar la conexión.

In [48]:
conexion.close()